In [1]:
!nvidia-smi

Sun Jan 29 19:23:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   34C    P0    45W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cudf
import glob
import gc

import pandas as pd

from tqdm import tqdm

In [4]:
def calc_recall(sub, ty='clicks', file='./data/xgb_train_y.parquet'):
    sub = sub.sort_values(['session', 'score'], ascending=[True, False])
    sub['dummy'] = 1
    sub['rank'] = sub.groupby(['session']).dummy.cumsum()
    sub.drop(['dummy'], axis=1, inplace=True)
    sub = sub[sub['rank']<21]
    sub = sub[['session', 'cand']]
    sub.columns = ['session', 'aid']
    test_labels = cudf.read_parquet(file)
    test_labels = test_labels[test_labels['type']==ty]
    test_labels = test_labels.merge(
        test_labels.groupby(['session', 'type']).count().reset_index().rename(columns={'aid': 'no_gt'}),
        how='left',
        on=['session', 'type']
    )
    sub['target'] = 1
    sub.columns = ['session', 'aid', 'target']
    test_labels = test_labels.merge(
        sub[['session', 'aid', 'target']].drop_duplicates(['session', 'aid']),
        how='left',
        on=['session', 'aid']

    )
    test_labels['target'] = test_labels['target'].fillna(0)
    test_labels = test_labels[test_labels['session'].isin(sub['session'])]
    test_labels = test_labels.groupby(['session', 'aid']).agg({'no_gt': 'min', 'target': 'sum'}).reset_index()
    recall = test_labels.target.sum()/test_labels.groupby(['session']).no_gt.min().clip(0,20).sum()
    print(recall)
    return(recall, test_labels.target.sum(), test_labels.groupby(['session']).no_gt.min().clip(0,20).sum())

In [5]:
!ls ./data/split/chunks

carts  clicks  orders


In [6]:
import xgboost as xgb

xgb_parms = { 
    'max_depth':8, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':0.3, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist' 
}

bl_sub = True
bl_pos = True
outputfolder = '24_XGB_Rerun_RedruceCandidate_DifferentWeights_Folds_ChrisCo_SameDay_v3_emb_mt'
no_seeds = 3
no_trees = 394
if not bl_sub:
    no_trees = 1000

In [7]:
ty = 'clicks'
labels = cudf.read_parquet('./data/xgb_train_y.parquet')
labels.columns = ['session', 'cand', 'type']
labels['target'] = 1

In [8]:
files = sorted(glob.glob('./data/split/chunks/' + ty + '/chunk*.parquet'))
files

['./data/split/chunks/clicks/chunk_0.parquet',
 './data/split/chunks/clicks/chunk_1.parquet']

In [9]:
[y for x in [
    sorted(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/chunks/' + ty + '/chunk*.parquet'))
    for igfold2 in range(0,5)
] for y in x ]

['./data_folds/fold_0/split/chunks/clicks/chunk_0.parquet',
 './data_folds/fold_0/split/chunks/clicks/chunk_1.parquet',
 './data_folds/fold_1/split/chunks/clicks/chunk_0.parquet',
 './data_folds/fold_1/split/chunks/clicks/chunk_1.parquet',
 './data_folds/fold_2/split/chunks/clicks/chunk_0.parquet',
 './data_folds/fold_2/split/chunks/clicks/chunk_1.parquet',
 './data_folds/fold_3/split/chunks/clicks/chunk_0.parquet',
 './data_folds/fold_3/split/chunks/clicks/chunk_1.parquet',
 './data_folds/fold_4/split/chunks/clicks/chunk_0.parquet',
 './data_folds/fold_4/split/chunks/clicks/chunk_1.parquet']

In [10]:
ingore_cols = []

In [11]:
%%time

if not bl_sub:
    # Local CV run
    # Predict each fold (total 10) by using 2 other folds
    model_bests = []
    hist_recall = []
    total_no_hit = 0
    total_no_gt = 0
    for igfold in range(5):
        files = sorted(glob.glob('./data_folds/fold_' + str(igfold) + '/split/chunks/' + ty + '/chunk*.parquet'))
        print(files)
        for ifile, file in enumerate(files):
            if bl_sub:
                if ty == 'clicks':
                    test_files = []
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
                elif ty == 'carts':
                    test_files = []
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
                elif ty == 'orders':
                    test_files = []
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
                else:
                    assert 0!=0
                if ty == 'clicks':
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
                elif ty == 'carts':
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
                elif ty == 'orders':
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
            else:
                if ty == 'clicks':
                    test_files = files[ifile]
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]][0:1]
                elif ty == 'carts':
                    test_files = files[ifile]
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]][0:1]
                elif ty == 'orders':
                    test_files = files[ifile]
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]][0:1]
                else:
                    assert 0!=0

                if ty == 'clicks':
                    train_files = [
                        y for x in [
                            sorted(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/chunks/' + ty + '/chunk*.parquet'))
                            for igfold2 in range(0,5)
                        ] for y in x if y not in test_files][0:2]
                else:
                    train_files = [
                        y for x in [
                            sorted(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/chunks/' + ty + '/chunk*.parquet'))
                            for igfold2 in range(0,5)
                        ] for y in x if y not in test_files]

                if ty == 'clicks':
                    df_test = pd.read_parquet(test_files)
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
                elif ty == 'carts':
                    df_test = pd.read_parquet(test_files)
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
                elif ty == 'orders':
                    df_test = pd.read_parquet(test_files)
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])

            print(train_files)
            print(test_files)

            if bl_pos and ty!='clicks':
                df_train = df_train[df_train['session'].isin(
                    df_train[df_train['target']==1]['session'].drop_duplicates().values
                )]

            print(df_train.shape)

            train_cols = [x for x in df_train.columns if x not in [
                'session', 'cand', 'target', 'target_clicks', 'target_carts', 'target_orders'
            ] + ingore_cols]

            print('Recall Train: ' + str(df_train[df_train['target']==1].shape[0]/labels[
            (labels['session'].isin(df_train['session'].drop_duplicates()))&(labels['type']==ty)
        ].shape[0]))
            if not bl_sub:
                print('Recall Test: ' + str(df_test[df_test['target']==1].shape[0]/labels[
                (labels['session'].isin(df_test['session'].drop_duplicates()))&(labels['type']==ty)
            ].shape[0]))

            dtrain = xgb.DMatrix(data=df_train[train_cols].values,label=df_train['target'].values)
            if not bl_sub:
                dtest =  xgb.DMatrix(data=df_test[train_cols].values, label=df_test['target'].values)
                df_test['score'] = 0.0

            del df_train
            gc.collect()

            models = []
            for iseed in range(no_seeds):
                print('iseed: ' + str(iseed))
                xgb_parms['seed'] = iseed
                if bl_sub:
                    model = xgb.train(
                        xgb_parms, 
                        dtrain=dtrain,
                        evals=[(dtrain,'train')],
                        num_boost_round=no_trees,
                        verbose_eval=10
                    )
                else:
                    model = xgb.train(
                        xgb_parms, 
                        dtrain=dtrain,
                        evals=[(dtest,'test')],
                        num_boost_round=no_trees,
                        verbose_eval=10,
                        early_stopping_rounds=50
                    )
                    df_test['score'] += model.predict(dtest)
                    model_bests.append(model.best_iteration)
                models.append(model)
            del dtrain

            if bl_sub:
                for isubfile, sub_file in enumerate(glob.glob('./data/split/sub/*.parquet')):
                    print('isubfile: ' + str(isubfile))
                    df_sub = cudf.read_parquet(sub_file)
                    df_sub['score'] = 0.0
                    dsub = xgb.DMatrix(data=df_sub[train_cols].values)
                    for iseed in range(no_seeds):
                        df_sub['score_' + str(iseed)] = models[iseed].predict(dsub)
                    iseed = 0
                    os.system('mkdir -p ../00_submissions/' + outputfolder)
                    df_sub[['session', 'cand'] + ['score_' + str(ij) for ij in range(no_seeds)]].to_parquet(
                        '../00_submissions/' + outputfolder + '/sub_' + str(iseed) + '_' + ty + '_' + str(ifile) + '_' + str(isubfile) + '.parquet'
                    )
                    del df_sub
                    gc.collect()
                del model
                gc.collect()
                break


            sub = cudf.from_pandas(df_test[['session', 'cand', 'score']])
            recall, no_hit, no_gt = calc_recall(sub, ty=ty)
            hist_recall.append(recall)
            total_no_hit += no_hit
            total_no_gt += no_gt
            print(total_no_hit/total_no_gt)

            del sub
            gc.collect()

['./data_folds/fold_0/split/chunks/clicks/chunk_0.parquet', './data_folds/fold_0/split/chunks/clicks/chunk_1.parquet']
['./data_folds/fold_0/split/chunks/clicks/chunk_1.parquet', './data_folds/fold_1/split/chunks/clicks/chunk_0.parquet']
./data_folds/fold_0/split/chunks/clicks/chunk_0.parquet
(31422066, 170)
Recall Train: 0.6351358350069342
Recall Test: 0.637195735843195
iseed: 0
[0]	test-logloss:0.60048
[10]	test-logloss:0.19091
[20]	test-logloss:0.08084
[30]	test-logloss:0.04522
[40]	test-logloss:0.03352
[50]	test-logloss:0.02972
[60]	test-logloss:0.02850
[70]	test-logloss:0.02809
[80]	test-logloss:0.02793
[90]	test-logloss:0.02784
[100]	test-logloss:0.02779
[110]	test-logloss:0.02776
[120]	test-logloss:0.02773
[130]	test-logloss:0.02770
[140]	test-logloss:0.02768
[150]	test-logloss:0.02767
[160]	test-logloss:0.02766
[170]	test-logloss:0.02765
[180]	test-logloss:0.02764
[190]	test-logloss:0.02763
[200]	test-logloss:0.02762
[210]	test-logloss:0.02762
[220]	test-logloss:0.02762
[230]	t

[450]	test-logloss:0.02745
[460]	test-logloss:0.02745
[470]	test-logloss:0.02745
[480]	test-logloss:0.02745
[489]	test-logloss:0.02745
0.565215389750141
0.5653382201905068
['./data_folds/fold_1/split/chunks/clicks/chunk_0.parquet', './data_folds/fold_1/split/chunks/clicks/chunk_1.parquet']
['./data_folds/fold_0/split/chunks/clicks/chunk_0.parquet', './data_folds/fold_0/split/chunks/clicks/chunk_1.parquet']
./data_folds/fold_1/split/chunks/clicks/chunk_0.parquet
(31424828, 170)
Recall Train: 0.6367653531799574
Recall Test: 0.6339367921922475
iseed: 0
[0]	test-logloss:0.60048
[10]	test-logloss:0.19090
[20]	test-logloss:0.08080
[30]	test-logloss:0.04518
[40]	test-logloss:0.03347
[50]	test-logloss:0.02967
[60]	test-logloss:0.02844
[70]	test-logloss:0.02803
[80]	test-logloss:0.02786
[90]	test-logloss:0.02778
[100]	test-logloss:0.02772
[110]	test-logloss:0.02769
[120]	test-logloss:0.02766
[130]	test-logloss:0.02764
[140]	test-logloss:0.02762
[150]	test-logloss:0.02761
[160]	test-logloss:0.02

[50]	test-logloss:0.02964
[60]	test-logloss:0.02842
[70]	test-logloss:0.02800
[80]	test-logloss:0.02783
[90]	test-logloss:0.02775
[100]	test-logloss:0.02769
[110]	test-logloss:0.02765
[120]	test-logloss:0.02762
[130]	test-logloss:0.02760
[140]	test-logloss:0.02758
[150]	test-logloss:0.02757
[160]	test-logloss:0.02755
[170]	test-logloss:0.02754
[180]	test-logloss:0.02753
[190]	test-logloss:0.02753
[200]	test-logloss:0.02752
[210]	test-logloss:0.02752
[220]	test-logloss:0.02751
[230]	test-logloss:0.02751
[240]	test-logloss:0.02751
[250]	test-logloss:0.02750
[260]	test-logloss:0.02750
[270]	test-logloss:0.02750
[280]	test-logloss:0.02750
[290]	test-logloss:0.02749
[300]	test-logloss:0.02749
[310]	test-logloss:0.02749
[320]	test-logloss:0.02749
[330]	test-logloss:0.02749
[340]	test-logloss:0.02749
[350]	test-logloss:0.02749
[360]	test-logloss:0.02749
[370]	test-logloss:0.02749
[380]	test-logloss:0.02749
[390]	test-logloss:0.02749
[400]	test-logloss:0.02749
[410]	test-logloss:0.02749
[420]	

[100]	test-logloss:0.02773
[110]	test-logloss:0.02769
[120]	test-logloss:0.02767
[130]	test-logloss:0.02764
[140]	test-logloss:0.02763
[150]	test-logloss:0.02761
[160]	test-logloss:0.02760
[170]	test-logloss:0.02759
[180]	test-logloss:0.02758
[190]	test-logloss:0.02757
[200]	test-logloss:0.02757
[210]	test-logloss:0.02756
[220]	test-logloss:0.02756
[230]	test-logloss:0.02755
[240]	test-logloss:0.02755
[250]	test-logloss:0.02755
[260]	test-logloss:0.02755
[270]	test-logloss:0.02754
[280]	test-logloss:0.02754
[290]	test-logloss:0.02754
[300]	test-logloss:0.02754
[310]	test-logloss:0.02754
[320]	test-logloss:0.02753
[330]	test-logloss:0.02753
[340]	test-logloss:0.02753
[350]	test-logloss:0.02753
[360]	test-logloss:0.02753
[370]	test-logloss:0.02753
[380]	test-logloss:0.02753
[390]	test-logloss:0.02753
[400]	test-logloss:0.02753
[410]	test-logloss:0.02753
[420]	test-logloss:0.02753
[421]	test-logloss:0.02753
iseed: 1
[0]	test-logloss:0.60048
[10]	test-logloss:0.19088
[20]	test-logloss:0.08

[130]	test-logloss:0.02756
[140]	test-logloss:0.02754
[150]	test-logloss:0.02753
[160]	test-logloss:0.02751
[170]	test-logloss:0.02750
[180]	test-logloss:0.02749
[190]	test-logloss:0.02749
[200]	test-logloss:0.02748
[210]	test-logloss:0.02747
[220]	test-logloss:0.02747
[230]	test-logloss:0.02747
[240]	test-logloss:0.02746
[250]	test-logloss:0.02746
[260]	test-logloss:0.02746
[270]	test-logloss:0.02746
[280]	test-logloss:0.02745
[290]	test-logloss:0.02745
[300]	test-logloss:0.02745
[310]	test-logloss:0.02745
[320]	test-logloss:0.02745
[330]	test-logloss:0.02745
[340]	test-logloss:0.02745
[350]	test-logloss:0.02745
[360]	test-logloss:0.02745
[370]	test-logloss:0.02745
[380]	test-logloss:0.02745
[390]	test-logloss:0.02745
[400]	test-logloss:0.02745
[410]	test-logloss:0.02745
[420]	test-logloss:0.02745
[428]	test-logloss:0.02744
iseed: 1
[0]	test-logloss:0.60047
[10]	test-logloss:0.19082
[20]	test-logloss:0.08067
[30]	test-logloss:0.04511
[40]	test-logloss:0.03334
[50]	test-logloss:0.02958

In [12]:
if not bl_sub:
    # Print average number of trees for submission
    import numpy as np
    np.mean(model_bests), np.std(model_bests), np.min(model_bests), np.max(model_bests)

(394.6, 26.30538601376785, 352, 453)

In [11]:
%%time

if bl_sub:
    # Submission
    # As the dataset is too large, only 2-folds can be loaded at once
    # There will be 5x models trained (10 folds // 2), each model has 3x bags and predict 5x submission datasets
    hist_recall = []
    total_no_hit = 0
    total_no_gt = 0
    for igfold in range(1):
        files = ['x', 'y', 'z', 'zz', 'zzz']
        print(files)
        for ifile, file in enumerate(files):
            if bl_sub:
                if ty == 'clicks':
                    test_files = []
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
                elif ty == 'carts':
                    test_files = []
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
                elif ty == 'orders':
                    test_files = []
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]]
                else:
                    assert 0!=0

                train_files = sorted([
                    y for x in [
                        sorted(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/chunks/' + ty + '/chunk*.parquet'))
                        for igfold2 in range(0,5)
                    ] for y in x
                ])

                step=2
                train_files_list = []
                for i in range(0, len(train_files), step):
                    x = i
                    train_files_list.append(train_files[x:x+step])
                train_files = train_files_list[ifile]
                print(train_files)

                if ty == 'clicks':
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
                elif ty == 'carts':
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
                elif ty == 'orders':
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
            else:
                if ty == 'clicks':
                    test_files = files[ifile]
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]][0:1]
                elif ty == 'carts':
                    test_files = files[ifile]
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]][0:1]
                elif ty == 'orders':
                    test_files = files[ifile]
                    train_files = [files[x] for x in [(ifile+i+1)%len(files) for i in range(len(files))]][0:1]
                else:
                    assert 0!=0

                train_files = [
                    y for x in [
                        sorted(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/chunks/' + ty + '/chunk*.parquet'))
                        for igfold2 in range(0,5)
                    ] for y in x if y not in test_files]

                if ty == 'clicks':
                    df_test = pd.read_parquet(test_files)
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])
                elif ty == 'carts':
                    df_test = cudf.read_parquet(test_files)
                    df_train = cudf.concat([
                        cudf.read_parquet(x) for x in train_files
                    ])
                elif ty == 'orders':
                    df_test = pd.read_parquet(test_files)
                    df_train = pd.concat([
                        pd.read_parquet(x) for x in train_files
                    ])

            print(train_files)
            print(test_files)

            if bl_pos:
                df_train = df_train[df_train['session'].isin(
                    df_train[df_train['target']==1]['session'].drop_duplicates().values
                )]

            train_cols = [x for x in df_train.columns if x not in [
                'session', 'cand', 'target', 'target_clicks', 'target_carts', 'target_orders'
            ] + ingore_cols]

            print('Recall Train: ' + str(df_train[df_train['target']==1].shape[0]/labels[
            (labels['session'].isin(df_train['session'].drop_duplicates()))&(labels['type']==ty)
        ].shape[0]))
            if not bl_sub:
                print('Recall Test: ' + str(df_test[df_test['target']==1].shape[0]/labels[
                (labels['session'].isin(df_test['session'].drop_duplicates()))&(labels['type']==ty)
            ].shape[0]))

            dtrain = xgb.DMatrix(data=df_train[train_cols].values,label=df_train['target'].values)
            if not bl_sub:
                dtest =  xgb.DMatrix(data=df_test[train_cols].values, label=df_test['target'].values)
                df_test['score'] = 0.0

            del df_train
            gc.collect()

            models = []
            for iseed in range(no_seeds):
                print('iseed: ' + str(iseed))
                xgb_parms['seed'] = iseed
                if bl_sub:
                    model = xgb.train(
                        xgb_parms, 
                        dtrain=dtrain,
                        evals=[(dtrain,'train')],
                        num_boost_round=no_trees,
                        verbose_eval=10
                    )
                else:
                    model = xgb.train(
                        xgb_parms, 
                        dtrain=dtrain,
                        evals=[(dtest,'test')],
                        num_boost_round=no_trees,
                        verbose_eval=10,
                        early_stopping_rounds=50
                    )
                    df_test['score'] += model.predict(dtest)
                models.append(model)
            del dtrain

            if bl_sub:
                for igfold2 in range(5):
                    for isubfile, sub_file in enumerate(glob.glob('./data_folds/fold_' + str(igfold2) + '/split/sub/*.parquet')):
                        print('isubfile: ' + str(isubfile))
                        df_sub = cudf.read_parquet(sub_file).fillna(-999)
                        df_sub['score'] = 0.0
                        dsub = xgb.DMatrix(data=df_sub[train_cols].values)
                        for iseed in range(no_seeds):
                            df_sub['score_' + str(iseed)] = models[iseed].predict(dsub)
                        iseed = 0
                        os.system('mkdir -p ../00_submissions/' + outputfolder)
                        df_sub[['session', 'cand'] + ['score_' + str(ij) for ij in range(no_seeds)]].to_parquet(
                            '../00_submissions/' + outputfolder + '/sub_' + str(igfold2) + '_' + ty + '_' + str(ifile) + '_' + str(isubfile) + '.parquet'
                        )
                        del df_sub
                        gc.collect()
                del model
                gc.collect()

            gc.collect()

['x', 'y', 'z', 'zz', 'zzz']
['./data_folds/fold_0/split/chunks/clicks/chunk_0.parquet', './data_folds/fold_0/split/chunks/clicks/chunk_1.parquet']
['./data_folds/fold_0/split/chunks/clicks/chunk_0.parquet', './data_folds/fold_0/split/chunks/clicks/chunk_1.parquet']
[]
Recall Train: 1.0
iseed: 0
[0]	train-logloss:0.60168
[10]	train-logloss:0.19793
[20]	train-logloss:0.09038
[30]	train-logloss:0.05611
[40]	train-logloss:0.04510
[50]	train-logloss:0.04158
[60]	train-logloss:0.04038
[70]	train-logloss:0.03990
[80]	train-logloss:0.03966
[90]	train-logloss:0.03949
[100]	train-logloss:0.03935
[110]	train-logloss:0.03923
[120]	train-logloss:0.03913
[130]	train-logloss:0.03903
[140]	train-logloss:0.03894
[150]	train-logloss:0.03886
[160]	train-logloss:0.03878
[170]	train-logloss:0.03871
[180]	train-logloss:0.03863
[190]	train-logloss:0.03856
[200]	train-logloss:0.03849
[210]	train-logloss:0.03844
[220]	train-logloss:0.03837
[230]	train-logloss:0.03832
[240]	train-logloss:0.03826
[250]	train-lo

[130]	train-logloss:0.03913
[140]	train-logloss:0.03904
[150]	train-logloss:0.03895
[160]	train-logloss:0.03887
[170]	train-logloss:0.03879
[180]	train-logloss:0.03871
[190]	train-logloss:0.03864
[200]	train-logloss:0.03858
[210]	train-logloss:0.03852
[220]	train-logloss:0.03846
[230]	train-logloss:0.03839
[240]	train-logloss:0.03833
[250]	train-logloss:0.03827
[260]	train-logloss:0.03821
[270]	train-logloss:0.03816
[280]	train-logloss:0.03811
[290]	train-logloss:0.03805
[300]	train-logloss:0.03800
[310]	train-logloss:0.03794
[320]	train-logloss:0.03789
[330]	train-logloss:0.03785
[340]	train-logloss:0.03780
[350]	train-logloss:0.03775
[360]	train-logloss:0.03769
[370]	train-logloss:0.03764
[380]	train-logloss:0.03758
[390]	train-logloss:0.03753
[393]	train-logloss:0.03751
isubfile: 0
isubfile: 1
isubfile: 2
isubfile: 3
isubfile: 4
isubfile: 5
isubfile: 6
isubfile: 7
isubfile: 8
isubfile: 9
isubfile: 10
isubfile: 11
isubfile: 12
isubfile: 13
isubfile: 14
isubfile: 15
isubfile: 16
isubf

[110]	train-logloss:0.03929
[120]	train-logloss:0.03918
[130]	train-logloss:0.03908
[140]	train-logloss:0.03899
[150]	train-logloss:0.03891
[160]	train-logloss:0.03883
[170]	train-logloss:0.03875
[180]	train-logloss:0.03868
[190]	train-logloss:0.03861
[200]	train-logloss:0.03855
[210]	train-logloss:0.03848
[220]	train-logloss:0.03841
[230]	train-logloss:0.03836
[240]	train-logloss:0.03829
[250]	train-logloss:0.03823
[260]	train-logloss:0.03818
[270]	train-logloss:0.03813
[280]	train-logloss:0.03807
[290]	train-logloss:0.03801
[300]	train-logloss:0.03795
[310]	train-logloss:0.03789
[320]	train-logloss:0.03784
[330]	train-logloss:0.03779
[340]	train-logloss:0.03774
[350]	train-logloss:0.03769
[360]	train-logloss:0.03765
[370]	train-logloss:0.03759
[380]	train-logloss:0.03754
[390]	train-logloss:0.03749
[393]	train-logloss:0.03747
iseed: 1
[0]	train-logloss:0.60168
[10]	train-logloss:0.19792
[20]	train-logloss:0.09031
[30]	train-logloss:0.05618
[40]	train-logloss:0.04511
[50]	train-loglos

[340]	train-logloss:0.03777
[350]	train-logloss:0.03772
[360]	train-logloss:0.03766
[370]	train-logloss:0.03761
[380]	train-logloss:0.03757
[390]	train-logloss:0.03752
[393]	train-logloss:0.03751
isubfile: 0
isubfile: 1
isubfile: 2
isubfile: 3
isubfile: 4
isubfile: 5
isubfile: 6
isubfile: 7
isubfile: 8
isubfile: 9
isubfile: 10
isubfile: 11
isubfile: 12
isubfile: 13
isubfile: 14
isubfile: 15
isubfile: 16
isubfile: 17
isubfile: 18
isubfile: 19
isubfile: 20
isubfile: 21
isubfile: 22
isubfile: 23
isubfile: 24
isubfile: 0
isubfile: 1
isubfile: 2
isubfile: 3
isubfile: 4
isubfile: 5
isubfile: 6
isubfile: 7
isubfile: 8
isubfile: 9
isubfile: 10
isubfile: 11
isubfile: 12
isubfile: 13
isubfile: 14
isubfile: 15
isubfile: 16
isubfile: 17
isubfile: 18
isubfile: 19
isubfile: 20
isubfile: 21
isubfile: 22
isubfile: 23
isubfile: 24
isubfile: 0
isubfile: 1
isubfile: 2
isubfile: 3
isubfile: 4
isubfile: 5
isubfile: 6
isubfile: 7
isubfile: 8
isubfile: 9
isubfile: 10
isubfile: 11
isubfile: 12
isubfile: 13
is